**1. MLP 구현**

In [ ]:
import torch.nn as nn

# 다층 퍼셉트론 이진 분류기 모델을 만들기 위해서는 nn.Linear 객체가 두 개 필요
fc1 = nn.Linear(3, 3) # 입력과 은닉을 연결
fc2 = nn.Linear(3, 2) # 은닉과 출력을 연결

print(fc1)

# Deep Learning 모델을 살필 때는 파라미터로 가중치랑 편향 확인
# 학습이 안 된 상태 - 초기화(랜덤) 값
print(fc1.weight)
print(fc1.bias)

Linear(in_features=3, out_features=3, bias=True)
Parameter containing:
tensor([[ 0.4148, -0.1421, -0.1588],
        [-0.2669, -0.0184,  0.2104],
        [ 0.0234, -0.0314, -0.1042]], requires_grad=True)
Parameter containing:
tensor([-0.1916,  0.3572,  0.0405], requires_grad=True)


In [ ]:
import torch

# 첫 번째 계층에 무작위 입력값 통과 (퍼셉트론 연산 수행)
x = torch.rand(3)
print("Input vector x")
print(x, "\n")

# 해당 계층의 변수 이름과 괄호를 통해 통과시키고자 하는 입력값을 전달
print("1st layer input")
print(fc1(x))

Input vector x
tensor([0.8588, 0.9737, 0.0774]) 

1st layer input
tensor([0.0139, 0.1264, 0.0220], grad_fn=<AddBackward0>)


In [ ]:
# 입력값의 수가 맞지 않는다면 계층이 처리하지 못해 오류 발생
x_error = torch.rand(4)
print("Wrong input vector")
print(x_error, "\n")

# 입력 벡터가 fc1의 가중치 벡터와 선형 연산(행렬곱)이 불가능한 형상이기 때문
print("Error on 1st layer")
print(fc1(x_error))

Wrong input vector
tensor([0.6698, 0.1871, 0.0650, 0.2628]) 

Error on 1st layer


RuntimeError: ignored

In [ ]:
# 미니배치 단위

# 미니배치 단위가 되면 입력 벡터가 입력 행렬이 되면서 차원이 2로 늘어남
# 중요한 건 in_feature=3 (4는 미니배치)
x_batch = torch.rand(4, 3)

# 행렬곱 -> 앞 행렬의 두 번째 차원, 뒤 행렬의 첫 번째 차원 값만 같으면 수행 가능
print("fc1 layer weight shape: {}".format(fc1.weight.shape))
print("fc1 layer bias shape: {}".format(fc1.bias.shape))
print("Input batch output shape: {}".format(fc1(x_batch).shape))

fc1 layer weight shape: torch.Size([3, 3])
fc1 layer bias shape: torch.Size([3])
Input batch output shape: torch.Size([4, 3])


In [ ]:
import torch.nn.functional as F

class MLP(nn.Module):
  # 첫번째 함수를 통해서 초기화
  def __init__(self):
    super().__init__() # (nn.Module)에 있는 것들을 다 활용. 초기화 단계. 관례적으로 같이 씀
    self.fc1 = nn.Linear(3, 3)
    self.fc2 = nn.Linear(3, 2)

  # forward 라는 함수명으로 층을 쌓음 (layer 간의 연결)
  def forward(self, x):
    x = self.fc1(x)
    x = torch.sigmoid(x)  # 계층 사이에 활성화 함수

    x = self.fc2(x)
    x = F.softmax(x, dim=-1) # 두 번째 계층 이후 소프트맥스 함수 적용

    return x

def forward : 층을 거쳐가면서 업데이트되는 식으로 처리   
입력변수 'x'  
- fc1을 거친 결과를 다시 x로 업데이트
- 비선형적 활섬화 함수(sigmoid)를 통해 업데이트  
- 은닉층에서 출력층으로 가는 fc2를 통해 업데이트  
- softmax를 통해 결과값을 확률로 나타냄

In [ ]:
model = MLP() # 객체 정의 안 해줬을 때 에러

x_batch = torch.rand(4, 3)
print("Model output")
print(model(x_batch))

Model output
tensor([[0.6888, 0.3112],
        [0.6896, 0.3104],
        [0.6896, 0.3104],
        [0.6903, 0.3097]], grad_fn=<SoftmaxBackward0>)


**2. 비용함수**

In [ ]:
print(model(x)) # 모델이 추정한 각 레이블에 대한 확률분포

tensor([0.6882, 0.3118], grad_fn=<SoftmaxBackward0>)


In [ ]:
criterion = nn.CrossEntropyLoss()

model_output = model(x).unsqueeze(0) # 배치 단위만 입력으로 받기 때문에 unsqueeze() 메소드로 "빈 차원" 하나를 배치 단위 자리에 추가
loss = criterion(model_output, torch.tensor([0]))
print(loss) # 계산되어 나온 오차 함수 값

# 함수 호출 한 번으로 역전파 알고리즘 수행
loss.backward()

# 모델의 첫 번째 퍼셉트론 계층 편향에 전달된 기울기값
model.fc1.bias.grad 

tensor(0.5225, grad_fn=<NllLossBackward0>)


tensor([-0.0026, -0.0274,  0.0168])